Let's start with basics: DNA code, codon tables and amino acids (extended with '_' for the STOP codon).

In [1]:
import numpy as np
from scipy.linalg import expm

# We use DNA codes (instead of RNA: T vs U) everywhere
NAindex = {'A':0, 'T':1, 'C':2, 'G':3}
codons = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }

# to later map codons to indices inside stochastic matrices
codon_index = {}
for i,c in enumerate(codons.keys()):
    codon_index[c] = i

AA_order = "IMTNKSRLPHQVADEGFYCW_" # codons order with STOP at the end

# to later map AAs to indices inside stochastic matrices
AA_index = {}
for i,a in enumerate(AA_order):
    AA_index[a] = i

We used https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7560444/ to find DNA mutation rates for CoVid, derived using MEGA (https://www.megasoftware.net/). The data needs to be 'normalized' (sum of rates should be 0). The "best" model for AIC or BIC seems the more flexible one (GTR).

In [2]:
def norm_rate(rate):
    d = np.sum(rate,axis=1)
    np.fill_diagonal(rate,-d)
    return rate
    
# GTR model
m_rate_GTR = norm_rate(np.array([[0 ,    12.05, 8.25, 8.97],
                                 [10.64,  0   , 6   , 3.64],
                                 [11.87,  9.78, 0   , 5.72],
                                 [12.13 , 5.57, 5.38, 0   ]]))

To compute AA to AA mutation frequencies, we will need to get codon biases. Instead of deriving them from the above, we used the RSCU values from https://virologyj.biomedcentral.com/articles/10.1186/s12985-020-01395-x. The RSCU is the ratio between observed and expected (uniform) probability among synonymous codons. Its sum over the codons of a given AA should be the code degeneracy for this AA. Because of rounding, this is not exactly true in the table below.

In [3]:
SARS_Cov_CU = {
'AGA': 2.67,
'TAA': 2.4,
'GGT': 2.34,
'GCT': 2.18,
'TCT': 1.97,
'GTT': 1.95,
'CCT': 1.94,
'ACT': 1.78,
'CTT': 1.74,
'TCA': 1.66,
'ACA': 1.64,
'TTA': 1.63,
'CCA': 1.59,
'TGT': 1.55,
'ATT': 1.52,
'CGT': 1.46,
'AGT': 1.44,
'GAA': 1.44,
'TTT': 1.41,
'CAT': 1.39,
'CAA': 1.39,
'AAT': 1.35,
'AAA': 1.31,
'GAT': 1.28,
'TAT': 1.22,
'GCA': 1.1,
'TTG': 1.07,
'ATG': 1.0,
'TGG': 1.0,
'ATA': 0.93,
'GTA': 0.91,
'GGA': 0.83,
'AGG': 0.81,
'TAC': 0.78,
'GGC': 0.72,
'GAC': 0.72,
'AAG': 0.69,
'CTA': 0.66,
'AAC': 0.65,
'CAG': 0.61,
'CAC': 0.61,
'CTC': 0.6,
'TTC': 0.59,
'CGC': 0.58,
'GTG': 0.57,
'GCC': 0.57,
'GAG': 0.56,
'GTC': 0.56,
'ATC': 0.56,
'TCC': 0.46,
'TGC': 0.45,
'ACC': 0.38,
'AGC': 0.36,
'TAG': 0.3,
'TGA': 0.3,
'CCC': 0.29,
'CTG': 0.29,
'CGA': 0.29,
'ACG': 0.2,
'CGG': 0.19,
'CCG': 0.17,
'GCG': 0.15,
'GGG': 0.12,
'TCG': 0.11
}

Now, the mutation rates can be transformed into transition probability matrices using matrix exponentiation (assuming a given time step `t`. We used scipy Pade approximation (see top import).

In [4]:
def rate2trans(rate,t):
    """converts a mutation rate matrix into a transition matrix for steps t"""
    return expm(rate*t)

We can now build a codon2codon transition probability matrix, assuming each codon position mutates independently of others (which should be Ok at DNA mutation level). This produces a stochastic matrix of conditional probabilities, each row summing to 1.

In [5]:
def codon2codon(rate,t):
    """Computes a codon to codon transition probability matrix for each pair of codons"""
    tp = rate2trans(rate,t)
    c2c = np.zeros((64,64))
    for i1,c1 in enumerate(codons.keys()):
        for i2,c2 in enumerate(codons.keys()):
            f = 1.0
            for i in range(0,3):
                f *= tp[NAindex[c1[i]],NAindex[c2[i]]]
            c2c[i1,i2] = f
    return c2c

# checking stochasticity
c2c = codon2codon(m_rate_GTR,1.0/1650)
print(np.sum(c2c,axis=1))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


We are now in good shape to compute our AA to AA transition probability matrix (again a stochastic matrix). We will assume that each AA is represented stochastically by a hiddden codon that appears with its codon bias frequency, given the AA. 

We need to gather the list of codons for each AA from the codon table above.

In [6]:
def aa2c():
    """Builts and returns a dictionary mapping each AA to its codon list"""
    aatoc = {}
    for c,a in codons.items():
        if a not in aatoc: aatoc[a] = []
        aatoc[a].append(c)
    return aatoc

We can compute the AA to AA transition matrix. We first correct the RSCU ratio so that they properly sum to the expected number of codons (this is slightly violated because of rounding to 2 decimals in the RSCU table above).  The probability of transition to a given amino acid `aa2` from an amino acid `aa1` is just the sum of all possible transitions over all codons representing these AAs, weighted by the codon bias of `aa1`.

This should again give a stochastic matrix of conditional probabilities.

In [9]:
def AA2AA(rate,t):
    """Computes an AA to AA transition probability matrix for each pair of 
    AAs (including STOP aka '_'). We assume that the hidden codon used by an 
    AA appears with it's codon bias marginal probability."""
    A2A = np.zeros((21,21))
    aa2cod = aa2c()
    # correct RSCU for rounding to preserve probabilities
    for aa,lcod in aa2cod.items():
        m = 0
        for cod in lcod:
            m += SARS_Cov_CU[cod]
        rm = m/float(len(lcod))
        for cod in lcod:
            SARS_Cov_CU[cod] /= rm

    cod2cod = codon2codon(rate,t)
    for aa1,lcod1 in aa2cod.items():
        ncod1 = len(lcod1)
        apcod1 = 1.0/ncod1 # a priori frequency without codon bias
        for cod1 in lcod1:
            fcod1 = apcod1 * SARS_Cov_CU[cod1] # codon bias corrected frequency 
            for aa2,lcod2 in aa2cod.items():
                for cod2 in lcod2:
                    f = fcod1 * cod2cod[codon_index[cod1],codon_index[cod2]]
                    A2A[AA_index[aa1],AA_index[aa2]] += f
    return A2A

# checking stochasticity
a2a = AA2AA(m_rate_GTR,1.0/(27*3*25))
print(np.sum(a2a,axis=1))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


We can transform this to an "energy" (log probability) matrix using a $-\log(\cdot)$ transform. We use a time step of $\frac{1}{27\times 3\times 25}$ time-step that should match the rate of one DNA mutation over 27 amino acids/codons. The table makes sense: the higher the number, the less likely the mutation because of DNA mutation rates, codon biases and codon table: `W` is hard to reach for example, mostly because it has just one codon. `W` also likes to mutate into a STOP codon because its codon - TGG - is very close to 2 stop codons - TGA and TAG - with just one mutation from G to A, while CoViD is AT rich and its polymerase loves mutating G into A.

In [10]:
a2a_energy = -np.log(a2a)
for i,a in enumerate(AA_order):
  print('   ',a, end = '')
print()
for i1,a1 in enumerate(AA_order):
  print(a1,end=' ')
  for i2,a2 in enumerate(AA_order):
    print(f'{a2a_energy[i1,i2]:4.1f}',end=' ')
  print()

# Less likely mutation
llm_from, llm_to = np.unravel_index(np.argmax(a2a_energy, axis=None), a2a_energy.shape)
print(f'\nLess likely mutation: {AA_order[llm_from]} to {AA_order[llm_to]}')

    I    M    T    N    K    S    R    L    P    H    Q    V    A    D    E    G    F    Y    C    W    _
I  0.0  5.9  5.8  5.6  6.4  6.7  7.5  5.1 11.3 11.1 11.9  5.4 11.3 11.1 11.9 11.8  5.5 10.8 11.8 17.3 11.3 
M  4.5  0.0  5.8 10.5  5.3 10.5  6.3  4.6 11.3 16.0 10.8  5.4 11.3 15.9 10.7 11.8 10.4 15.6 16.7 11.5 10.4 
T  5.4  8.3  0.0  5.8  5.9  4.9  6.7 10.3  5.5 11.3 11.4 10.8  5.4 11.2 11.4 11.3 11.1 10.9 11.6 14.0 10.7 
N  5.2 11.3  5.5  0.0  4.9  5.4  9.9 10.6 11.0  5.5 10.4 10.6 10.9  5.5 10.3 10.9 10.3  5.2 10.6 16.7 10.0 
K  5.6  6.2  5.5  4.8  0.0  9.7  5.4  9.8 11.0 10.3  5.5 10.6 10.9 10.2  5.5 10.9 15.1  9.9 15.3 11.6  5.2 
S  7.1 13.1  5.4  6.3 10.7  0.0  5.7  6.6  6.2 11.2 12.2 11.6  6.7 11.3 12.7  6.6  6.3  6.1  5.9  9.7  6.0 
R  6.7  7.9  6.5 10.1  5.7  5.0  0.0  6.8  6.8  6.2  7.6 11.5 11.5 12.1 11.0  5.6 12.3 11.5  6.4  7.0  5.9 
L  5.5  6.7 11.0 11.3 11.0  6.6  6.9  0.0  6.4  6.2  7.1  6.1 11.9 12.1 11.9 12.4  5.2 10.5 11.5  8.1  5.9 
P 10.5 13.6  5.2 10.9 11.1  5.